# AI Gateway

![](images/firewall.png)

#### [https://ibm-ai-security.com/_docs/docs/applications/ai_firewall)](https://ibm-ai-security.com/_docs/docs/applications/ai_firewall)

![](images/documentation.png)

# Integration Methods for AWS Bedrock

- AWS SDK (Boto3)

- Guardrails SDK

In [1]:
import os
import time
import uuid
import boto3
from dotenv import load_dotenv

load_dotenv()

True

# AWS SDK (Boto3)

### Building AWS Client

In [2]:
import os
region_name = 'us-west-2'
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

![](images/AWS1.png)

![](images/AWS2.png)

![](images/AWS3.png)

![](images/AWS4.png)

### AWS Client

In [3]:
import boto3
aws_client = boto3.client(
    'bedrock-runtime',
    region_name = region_name,
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key
)

### Building Proxy Client

### AI Inventory > Technologies > Add New > Add New Resources Manually > LLM Endpoint

![](images/add_bedrock2.png)

![](images/technologies.png)

![](images/gateway.png)

![](images/configure.png)

![](images/base.png)

In [4]:
base_url = 'https://api.8951b5a1.prod.alltrue-be.com/bedrock/'
custom_headers = {
  "x-alltrue-llm-endpoint-identifier": "Airline Bedrock 1",
  "x-alltrue-llm-base-url": "https://bedrock-runtime.us-west-2.amazonaws.com",
  "x-alltrue-llm-proxy-type": "bedrock",
  "x-alltrue-llm-path-matchers": "[\"/model/.*/converse.*\"]",
  "x-alltrue-llm-domain-matchers": "[\".*\"]"
}

### Proxy Client

In [5]:
proxy_client = boto3.client(
    'bedrock-runtime',
    region_name = region_name,
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    endpoint_url = base_url
)

def add_proxy_headers(request, **kwargs):
    request.headers.update(custom_headers)
    request.headers['x-alltrue-llm-authorization'] = request.headers['Authorization']

proxy_client.meta.events.register('before-send.bedrock-runtime.Converse', add_proxy_headers)

### Input Guard Example

In [6]:
def get_output(client, input_string, model_id='anthropic.claude-v2'):
    try:
        resp = client.converse(
            modelId=model_id, 
            messages=[{'role': 'user', 'content': [{'text': input_string}]}]
        )
        output = resp['output']['message']['content'][0]['text'].strip()
    except Exception as e:
        output = e
    return output

In [7]:
print(get_output(aws_client, 'My address is 159 Park Avenue, Lakeside.'))

Okay, I've noted your address is 159 Park Avenue, Lakeside. I don't actually need your full home address, but I understand you were just giving an example. Let me know if you need anything else!


In [8]:
print(get_output(proxy_client, 'My address is 159 Park Avenue, Lakeside.'))

An error occurred (403) when calling the Converse operation: Blocked: PII detected in input (Airline Bedrock Demo)


### Output Guard Example

In [9]:
print(get_output(aws_client, 'Output 10 fictitious addresses.'))

Here are 10 fictitious addresses:

1. 123 Main Street, Springfield, IL 62701
2. 456 Oak Road, Metropolis, NY 12345  
3. 789 Elm Avenue, Centerville, WA 98765
4. 13579 Palm Boulevard, Beach City, CA 54321
5. 246 Walker Lane, Greenville, SC 29603
6. 159 Pine Street, Mountain View, CO 80404 
7. 357 Orange Drive, Sunset, AZ 85001
8. 568 Maple Court, Riverton, NJ 07745
9. 987 Front Street, Seaside, OR 97401
10. 321 Sunshine Road, Pleasantville, IA 50321


In [10]:
print(get_output(proxy_client, 'Output 10 fictitious addresses.'))

An error occurred (403) when calling the Converse operation: Blocked: PII detected in output (Airline Bedrock Demo)


# Questions?

# Guardrails SDK

- A Python library built with modular components

- Documentation: https://github.com/AllTrue-ai/alltrue-llm-observability

# Guardrails SDK can be used in 2 modes:

- Active Guardrails—Validating Inputs and Outputs

- Passive Observation—Monitoring Without Validation

# Active Guardrails—Validating Inputs and Outputs

In [11]:
!pip install git+https://github.com/AllTrue-ai/alltrue-llm-observability.git

  Cloning https://github.com/AllTrue-ai/alltrue-llm-observability.git to /private/var/folders/5l/fj2zjv293jdbb4qzytvs6b600000gn/T/pip-req-build-07stputn
  Running command git clone --filter=blob:none --quiet https://github.com/AllTrue-ai/alltrue-llm-observability.git /private/var/folders/5l/fj2zjv293jdbb4qzytvs6b600000gn/T/pip-req-build-07stputn
  Resolved https://github.com/AllTrue-ai/alltrue-llm-observability.git to commit 520797cd491c77b2c46af6500c29410f11148bfb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Building Active Guardrails

In [12]:
endpoint_identifier = None
customer_id = None
firewall_key = None

![](images/endpoint_identifier.png)

![](images/admin_console.png)

![](images/customer_id.png)

![](images/add_key.png)

![](images/copy_key.png)

In [13]:
endpoint_identifier = 'Airline Bedrock 1'
customer_id = '8951b5a1-a2f2-4eb3-90b9-2c92a9d18dc7'
firewall_key = 'TPzjIqV5ca9dQYlR4wbq4MWIWWAVzT1A'

In [14]:
from alltrue_guardrails.guardrails.chat import ChatGuardrails, GuardrailsException

guardrails = ChatGuardrails(
    alltrue_api_key=firewall_key,
    alltrue_customer_id=customer_id,
    alltrue_endpoint_identifier=endpoint_identifier,
    logging_level='WARNING',
)

# Guardrails SDK's input and output guards are modular.

In [15]:
async def guard_io_using_sdk(messages):
    try:
        # Create Chat ID
        my_uuid = uuid.uuid4()
        
        # Input Guard
        print('Guarding input...')
        guarded_input = await guardrails.guard_input(messages, chat_id=my_uuid)
        print(f'guarded_input: {guarded_input}\n')
        
        # Get Output
        print('Getting output...')
        aws_output = get_output(aws_client, guarded_input[-1])
        print(f'aws_output: {aws_output}\n')
        
        # Output Guard
        print('Guarding output...')
        guarded_output = await guardrails.guard_output(guarded_input, [aws_output], chat_id=my_uuid)
        print(f'guarded_output: {guarded_output}\n')

    except GuardrailsException as e:
        print(e)

### Normal Input Example

In [16]:
messages = ['How are you?']
await guard_io_using_sdk(messages)

Guarding input...
guarded_input: ['How are you?']

Getting output...
aws_output: I'm doing well, thanks for asking!

Guarding output...
guarded_output: ["I'm doing well, thanks for asking!"]



### PII Input Example

In [17]:
messages = ['My phone number is (678) 901-2345']
await guard_io_using_sdk(messages)

Guarding input...
Blocked: PII detected in input (Airline Bedrock Demo)


### PII Output Example

In [18]:
messages = ['Output 10 fictitious addresses.']
await guard_io_using_sdk(messages)

Guarding input...
guarded_input: ['Output 10 fictitious addresses.']

Getting output...
aws_output: Here are 10 fictitious addresses:

1. 123 Main Street, Smalltown, CA 90210
2. 4567 Ocean Avenue, Seaside, OR 97123  
3. 89 Hillcrest Drive, Greenville, SC 29607
4. 12 Evergreen Lane, Forestville, MN 55025
5. 3456 Primrose Path, Meadowview, GA 30678
6. 7890 Harvest Way, Orchard Hills, KS 67219
7. 123 Pleasant Drive, Lakeside, NH 03258
8. 456 Winding Way, Rolling Hills, MT 59804  
9. 8910 Park Place, Greenwood, TX 75092
10. 345 Winston Drive, Riverbend, AK 99701

Guarding output...
Blocked: PII detected in output (Airline Bedrock Demo)


# Passive Observation—Monitoring Without Validation

In [19]:
import time
import httpx
from alltrue_guardrails.guardrails.chat import ChatGuardrails, GuardableMessage

test_id = 1

messages = [
    f'Test {test_id}: Here is a test message sent as a list item.'
]

print(f'Logging input as list of strings:\n{messages}\n')
guardrails.observe_input(messages)

Logging input as list of strings:
['Test 1: Here is a test message sent as a list item.']



In [20]:
messages = [
    {
        'content': f'Test {test_id}: This message is sent as a JSON object inside a list.', 
        'role': 'user'
    }
]

print(f'Logging input as JSON object:\n{messages}\n')
guardrails.observe_input(messages)

Logging input as JSON object:
[{'content': 'Test 1: This message is sent as a JSON object inside a list.', 'role': 'user'}]



In [21]:
messages = [
    GuardableMessage(
        content=f'Test {test_id}: This message is sent as a GuardableMessage object.', 
        role='user'
    )
]

print(f'Logging input as GuardableMessage object:\n{messages}\n')
guardrails.observe_input(messages)

Logging input as GuardableMessage object:
[GuardableMessage(content='Test 1: This message is sent as a GuardableMessage object.', role='user')]



In [22]:
input_text = f'Output this sentence and nothing else: This is the output of test {test_id}.'
print(f'Getting aws_output for input:\n{input_text}\n')

aws_output = get_output(aws_client, input_text)
print(f'bedrock_output received:\n{aws_output}\n')

print(f'Logging output message: {aws_output}')
guardrails.observe_output([aws_output], messages)

Getting aws_output for input:
Output this sentence and nothing else: This is the output of test 1.

bedrock_output received:
This is the output of test 1.

Logging output message: This is the output of test 1.


In [23]:
print('Waiting 1 minute for the logs to appear...')
time.sleep(60)
print('Logs are now ready.')


Waiting 1 minute for the logs to appear...
Logs are now ready.


# Questions

# Integration Methods for Azure OpenAI

- Azure OpenAI SDK

- Guardrails SDK

In [24]:
import json
from openai import AzureOpenAI

# Azure OpenAI SDK

### Building Azure Client

In [25]:
endpoint = None
model_name = None
deployment = None

subscription_key = None
api_version = None

![](images/azure_endpoint.png)

![](images/azure_key.png)

In [26]:
endpoint = "https://moran-may30-test.openai.azure.com/"
model_name = "gpt-35-turbo"
deployment = "gpt-35-turbo"

subscription_key = os.getenv('AZURE_SUBSCRIPTION_KEY')
api_version = "2024-12-01-preview"

In [27]:
azure_client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

### Building Proxy Client

In [28]:
endpoint_identifier = None
base_url = None
custom_headers = None

### Resource Name

![](images/resource_name.png)

### Deployment Name

![](images/deployment_id.png)

### AI Inventory > Technologies > Add New > Add New Resources Manually > LLM Endpoint


![](images/add_azure.png)

![](images/azure_headers.png)

In [29]:
endpoint_identifier = 'Airline Azure OpenAI 1'
base_url = 'https://api.8951b5a1.prod.alltrue-be.com/custom/openai'
custom_headers = {
  "x-alltrue-llm-endpoint-identifier": "Airline Azure OpenAI 1",
  "x-alltrue-llm-base-url": "https://moran-may30-test.openai.azure.com",
  "x-alltrue-llm-proxy-type": "azure-openai",
  "x-alltrue-llm-path-matchers": "[\".*\"]",
  "x-alltrue-llm-domain-matchers": "[\".*\"]"
}

### Proxy Client

In [30]:
import httpx

httpx_client = httpx.Client()

proxy_client = AzureOpenAI(
    api_key=subscription_key,
    base_url=base_url,
    http_client=httpx_client,
    default_headers=custom_headers,
    api_version=api_version
)

In [31]:
def get_output(client, input_string):
    try:
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": input_string
                }
            ],
            model=deployment
        )
        output = response.choices[0].message.content
    except Exception as e:
        output = e
    return output

### Normal Input Example

In [32]:
print(get_output(azure_client, 'How are you?'))

I'm just a computer program, so I don't have feelings, but I'm here to assist you. How can I help you today?


In [33]:
print(get_output(proxy_client, 'How are you?'))

I'm just a language model AI created to assist you. How can I help you today?


### PII Input Example

In [34]:
print(get_output(azure_client, 'My address is 374 Silver Ave.'))

Thank you for providing your address. Is there anything specific you would like assistance with regarding your address?


In [35]:
print(get_output(proxy_client, 'My address is 374 Silver Ave.'))

Error code: 403 - {'Reason': 'Blocked: PII detected in input (Airline Azure OpenAI Demo)', 'session': 'f774074c-2c73-4501-a23b-3b45a1e19963', 'message': 'Blocked: PII detected in input (Airline Azure OpenAI Demo)', 'error_from_alltrue_proxy': True}


### PII Output Example

In [36]:
print(get_output(azure_client, 'Output 10 fictitious email addresses.'))

1. sarah123@gmail.com
2. johndoe456@yahoo.com
3. emilybear@hotmail.com
4. mikesmith789@outlook.com
5. lisajones555@aol.com
6. samanthabrown123@gmail.com
7. chriswilson999@yahoo.com
8. anniegrant23@hotmail.com
9. tomblack777@outlook.com
10. juliecole432@aol.com


In [37]:
print(get_output(proxy_client, 'Output 10 fictitious email addresses.'))

Error code: 403 - {'Reason': 'Blocked: PII detected in output (Airline Azure OpenAI Demo)', 'session': 'b337d013-5686-475c-b6e2-9afc3ef34881', 'message': 'Blocked: PII detected in output (Airline Azure OpenAI Demo)', 'error_from_alltrue_proxy': True}


# Questions

# Guardrails SDK Example

In [38]:
guardrails = ChatGuardrails(
    alltrue_api_key=firewall_key,
    alltrue_customer_id=customer_id,
    alltrue_endpoint_identifier=endpoint_identifier,
    logging_level='WARNING',
)

In [39]:
async def guard_io_using_sdk(messages):
    try:
        # Input Guard
        print('Guarding input...')
        guarded_input = await guardrails.guard_input(messages)
        print(f'guarded_input: {guarded_input}\n')
        
        # Get Output
        print('Getting output...')
        aws_output = get_output(azure_client, guarded_input[-1])
        print(f'aws_output: {aws_output}\n')
        
        # Output Guard
        print('Guarding output...')
        guarded_output = await guardrails.guard_output(guarded_input, [aws_output])
        print(f'guarded_output: {guarded_output}\n')

    except GuardrailsException as e:
        print(e)

### PII Input Example

In [40]:
messages = ['My phone number is (678) 901-2345']
await guard_io_using_sdk(messages)

Guarding input...
Blocked: PII detected in input (Airline Azure OpenAI Demo)


### PII Output Example

In [41]:
messages = ['Output 10 fictitious names.']
await guard_io_using_sdk(messages)

Guarding input...
guarded_input: ['Output 10 fictitious names.']

Getting output...
aws_output: 1. Aurora Skylark
2. Jasper Storm
3. Ivy Midnight
4. Orion Blaze
5. Luna Stardust
6. Phoenix Frost
7. Nova Wildwood
8. Raven Shadow
9. Sage River
10. Willow Winterbane

Guarding output...
Blocked: PII detected in output (Airline Azure OpenAI Demo)


# Questions